# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [4]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [5]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [6]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010, driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [7]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [8]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [9]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [10]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [11]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [33]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [12]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [24]:
!pip install panel
!pip install --upgrade openai

  Using cached openai-1.83.0-py3-none-any.whl.metadata (25 kB)
Using cached openai-1.83.0-py3-none-any.whl (723 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [25]:
import panel as pn
import openai
import os

pn.extension()

# Set your OpenAI API key
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # Secure way
# client = OpenAI(api_key="your-api-key-here")  # For quick testing

# Initialize chat history
chat_history = pn.pane.Markdown("**Bot**: Hi there! Want to hear a joke? (Type 'yes' or 'no')", width=500)
user_input = pn.widgets.TextInput(placeholder='Type here...', width=500)

# Function to fetch jokes
def get_jokes():
    try:
        response = client.chat.completions.create(
            model="gpt-4",  # or "gpt-3.5-turbo"
            messages=[
                {"role": "system", "content": "You're a funny assistant."},
                {"role": "user", "content": "Tell me 3 short jokes."}
            ],
            temperature=0.8
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Oops! Couldn't fetch jokes. Error: {e}"

# Callback to handle user input
def respond(event):
    user_message = user_input.value.strip().lower()
    chat_log = chat_history.object + f"\n\n**You**: {user_input.value}"

    if user_message in ['yes', 'y', 'sure', 'ok']:
        chat_log += "\n\n**Bot**: Awesome! Here's something to make you smile:\n"
        jokes = get_jokes()
        chat_log += f"{jokes}"
    elif user_message in ['no', 'n', 'nope']:
        chat_log += "\n\n**Bot**: No problem! Let me know when you're in the mood."
    else:
        chat_log += "\n\n**Bot**: Please type 'yes' if you want to hear a joke or 'no' to skip."

    chat_history.object = chat_log
    user_input.value = ""

user_input.param.watch(respond, 'value', onlychanged=True)

# Layout
chatbot_panel = pn.Column(chat_history, user_input)
chatbot_panel.servable()
chatbot_panel

BokehModel(combine_events=True, render_bundle={'docs_json': {'2b284ea8-c215-41bd-a7e2-01cb4ced2880': {'version…

## Report: Interactive Joke Bot with OpenAI and Panel

### Overview

In this notebook, we implemented an interactive joke bot using OpenAI's GPT models and the Panel library for UI. The bot greets the user and offers to tell jokes based on user input. If the user responds positively, the bot fetches and displays three short jokes using the OpenAI API.

### Findings

#### Variations and Observations

- **Positive Responses:** The bot successfully recognizes various affirmative responses such as "yes", "y", "sure", and "ok". Upon receiving these, it fetches jokes from the model and displays them in the chat history.
- **Negative Responses:** Inputs like "no", "n", and "nope" are correctly interpreted, and the bot politely acknowledges the user's choice.
- **Unexpected Inputs:** When the user enters something outside the expected responses, the bot prompts the user to reply with "yes" or "no". This helps guide the interaction and keeps the conversation on track.
- **Joke Quality:** The jokes returned by the model are generally appropriate, short, and humorous. However, there is some repetition if the function is called multiple times in quick succession, as the model may return similar jokes.
- **Error Handling:** If the API call fails, the bot displays an error message, ensuring the user is informed about the issue.

#### Issues and Hallucinations

- **Hallucination:** There were no significant hallucinations observed in the jokes themselves, as the prompt is simple and the model is well-suited for this task.
- **Formatting:** Occasionally, the jokes may not be separated clearly, depending on the model's output formatting. This can be improved by post-processing the response.
- **User Input Edge Cases:** If the user enters a long sentence or an unrelated question, the bot does not attempt to answer but instead repeats its prompt, which is a safe fallback but could be improved for a more natural conversation.

### Lessons Learned

- **Prompt Engineering:** Clear and specific prompts yield better results. Asking for "3 short jokes" consistently produces concise and relevant responses.
- **User Experience:** Handling a variety of user inputs (affirmative, negative, and unexpected) is crucial for a smooth conversational experience.
- **Model Strengths:** GPT models excel at generating creative content like jokes, especially when the prompt is well-defined.
- **Panel Integration:** Panel provides an easy way to build interactive UIs in Jupyter, making it suitable for rapid prototyping of chatbots and similar applications.
- **Error Handling:** Including try-except blocks for API calls ensures the application remains robust and user-friendly, even when external services fail.

### Conclusion

The joke bot demonstrates how combining OpenAI's language models with interactive Python libraries can create engaging user experiences. While the model performs well for this use case, careful prompt design and input handling are essential for reliability and user satisfaction. Future improvements could include more nuanced conversation handling and enhanced formatting of the model's responses.